"""
Generate data with specified changepoints
Compare ELBO fits for changepoint model with different
numbers of changepoints
"""

In [15]:
import os
import pickle
import scipy.stats as stats
import pymc3 as pm
import theano.tensor as tt
import numpy as np
import pylab as plt
from pymc3.variational.callbacks import CheckParametersConvergence
from theano.tensor.extra_ops import cumsum
import sys
sys.path.append('/media/bigdata/firing_space_plot/ephys_data')
from ephys_data import ephys_data
import visualize
import pandas as pd

sys.path.append('/media/bigdata/projects/pytau')
import pytau

import itertools as it
from tqdm import tqdm

In [2]:
nrn_num = 10
trial_num = 30
time_bins = 100
switch_components = 3
states = 4

In [10]:
true_tau = np.cumsum(np.random.random((trial_num,states)),axis=-1)
true_tau /= np.max(true_tau,axis=-1)[...,np.newaxis]
true_tau *= time_bins
true_tau = np.vectorize(np.int)(true_tau)

trial_tau = [8,23]
trial_lambda_selector = np.ones(trial_num)*len(trial_tau)
for val in trial_tau:
    trial_lambda_selector[:val] = trial_lambda_selector[:val]-1
trial_lambda_selector = np.vectorize(np.int)(trial_lambda_selector)

state_inds = np.concatenate([np.zeros((trial_num,1)),true_tau],axis=-1)
state_inds = np.vectorize(np.int)(state_inds)
true_tau = true_tau[:,:-1]

true_lambda = np.random.random((nrn_num,switch_components, states))

true_r = np.zeros((trial_num, nrn_num, time_bins))

for trial_ind in range(trial_num):
    for state_num in range(states):
        true_r[trial_ind,:,state_inds[trial_ind,state_num]:state_inds[trial_ind,state_num+1]] = \
                        true_lambda[:,trial_lambda_selector[trial_ind],state_num][:,np.newaxis]
        
spike_array = np.random.random(true_r.shape) < true_r  

In [11]:
state_list = range(2,7)
comps_list = range(2,5)
iters = list(it.product(state_list, comps_list))

In [12]:
elbo_list = []
fit = 40000
samples = 20000

In [13]:
#single_taste_poisson_trial_switch(spike_array,switch_components,states)
for this_iter in tqdm(iters):
    model = pytau.changepoint_model.single_taste_poisson_trial_switch(spike_array, this_iter[1], this_iter[0])
    with model:
        inference = pm.ADVI('full-rank')
        approx = pm.fit(n=fit, method=inference)
        #trace = approx.sample(draws=samples)
        elbo_list.append(-approx.hist[-1])

  0%|          | 0/15 [00:00<?, ?it/s]

Finished [100%]: Average Loss = 24,965
  7%|▋         | 1/15 [01:23<19:35, 84.00s/it]

Finished [100%]: Average Loss = 24,561
 13%|█▎        | 2/15 [02:36<17:26, 80.47s/it]

Finished [100%]: Average Loss = 24,664
 20%|██        | 3/15 [03:44<15:22, 76.86s/it]

Finished [100%]: Average Loss = 24,457
 27%|██▋       | 4/15 [04:59<13:57, 76.12s/it]

...

Finished [100%]: Average Loss = 23,729
 73%|███████▎  | 11/15 [14:18<05:29, 82.41s/it]

Finished [100%]: Average Loss = 23,761
 80%|████████  | 12/15 [15:47<04:12, 84.16s/it]

Finished [100%]: Average Loss = 24,213
 87%|████████▋ | 13/15 [17:23<02:55, 87.89s/it]

Finished [100%]: Average Loss = 23,818
 93%|█████████▎| 14/15 [18:59<01:30, 90.33s/it]

Finished [100%]: Average Loss = 23,857
100%|██████████| 15/15 [20:34<00:00, 82.31s/it]


In [16]:
param_array = np.array(iters)
elbo_frame = pd.DataFrame(dict(
                states = param_array[:,0],
                components = param_array[:,1],
                elbo = elbo_list))

## ==== Models with Top 3 ELBO ====

In [17]:
elbo_frame.sort_values('elbo', ascending=False).head(3)

,states,components,elbo
7,4,3,-23639.095514
8,4,4,-23677.130023
10,5,3,-23742.348086
